# Evaluate Well Stain Relationship Strength
This analysis compares Maximal Information Coefficient (MIC) scores between the same DAPI and nuclear speckle stain (A647 or GOLD) aggregated well features per well.
Distributions of these MIC scores are visualized, between zero and one, where one indicates a perfect relationship and zero indicates no relationship.
From this analysis we can underastand the relationships between the DAPI and nuclear speckle stains per cell population (Treatment, Well Position, Plate, etc...)
With these insights we may further improve or stain feature regression and stain translation models.
We save additional experimental details in a manifest for further evaluation.

In [1]:
import pathlib
import re
import sys

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

## Find the root of the git repo on the host system

In [2]:
# Get the current working directory
cwd = pathlib.Path.cwd()

if (cwd / ".git").is_dir():
    root_dir = cwd

else:
    root_dir = None
    for parent in cwd.parents:
        if (parent / ".git").is_dir():
            root_dir = parent
            break

# Check if a Git root directory was found
if root_dir is None:
    raise FileNotFoundError("No Git root directory found.")

# Custom Imports

In [3]:
sys.path.append(f"{root_dir}/0.data_analysis_and_processing/utils")

from MIC import MIC
from PairwiseCompare import PairwiseCompare
from ShuffledMIC import ShuffledMIC

# Inputs

In [4]:
# Paths to original nuclear speckle data
data_dir = root_dir / "nuclear_speckles_data"
nuclear_mask_dir = (data_dir / "Nuclear_masks").resolve(strict=True)
sc_profiles_path = list((data_dir / "Preprocessed_data/single_cell_profiles").resolve(strict=True).glob("*feature_selected*.parquet"))

# Load single-cell profile data
scdfs = [pd.read_parquet(sc_path) for sc_path in sc_profiles_path if sc_path.is_file()]

# Outputs

In [5]:
distribution_figures_path = pathlib.Path("well_sirna_mic_distribution_figures")
distribution_figures_path.mkdir(parents=True, exist_ok=True)

mic_comparisons_path = pathlib.Path("mic_comparisons_data")
mic_comparisons_path.mkdir(parents=True, exist_ok=True)

# Processing

## Combine Common Data
Column names are used to combine common single-cell data.

In [6]:
common_columns = scdfs[0].columns
for scdf in scdfs[1:]:
    common_columns = common_columns.intersection(scdf.columns)

scdfs = pd.concat(scdfs, axis=0)[common_columns]

In [7]:
scdfs.dropna(inplace=True)

print(scdfs)

      Metadata_CellLine Metadata_Condition  Metadata_ImageNumber  \
0                  786O                NTC                     1   
1                  786O                NTC                     1   
2                  786O                NTC                     1   
3                  786O                NTC                     1   
4                  786O                NTC                     1   
...                 ...                ...                   ...   
63354              293T          untreated                   407   
63355              293T          untreated                   407   
63356              293T          untreated                   407   
63357              293T          untreated                   407   
63358              293T          untreated                   407   

      Metadata_Plate Metadata_Well Metadata_Site  Metadata_Nuclei_Site_Count  \
0             slide3            A1           M14                          40   
1             slide3   

## Seperate Gold, A647, and Dapi Features

In [8]:
gold_scdfs = scdfs.loc[:, scdfs.columns.str.contains("GOLD|Metadata", regex=True)]
a647_scdfs = scdfs.loc[:, scdfs.columns.str.contains("A647|Metadata", regex=True)]
dapi_scdfs = scdfs.loc[:, scdfs.columns.str.contains("DAPI|Metadata", regex=True)]

gold_scdfs.columns = gold_scdfs.columns.str.replace('_GOLD', '', regex=False)
a647_scdfs.columns = gold_scdfs.columns.str.replace('_A647', '', regex=False)
dapi_scdfs.columns = dapi_scdfs.columns.str.replace('_DAPI', '', regex=False)

## Combine Seperated Stain Features

In [9]:
gold_scdfs = gold_scdfs.assign(Metadata_Stain="GOLD")
dapi_scdfs = dapi_scdfs.assign(Metadata_Stain="DAPI")
a647_scdfs = a647_scdfs.assign(Metadata_Stain="A647")

common_cols = gold_scdfs.columns.intersection(dapi_scdfs.columns).intersection(a647_scdfs.columns)
scdfs = pd.concat([gold_scdfs[common_cols], dapi_scdfs[common_cols], a647_scdfs[common_cols]], axis=0)

## Specify Feature Metadata Columns

In [10]:
feat_cols = scdfs.columns[~scdfs.columns.str.contains("Metadata")]

## Mean Aggregation to the Well Level

In [11]:
# Define Placeholder Column
scdfs["Metadata_Cell_Count"] = scdfs["Metadata_Condition"].values

# Metadata to retain
agg_funcs = {
    "Metadata_Condition": "first",
    "Metadata_Cell_Count": "size"
}

agg_funcs |= {feat_col: "mean" for feat_col in feat_cols}
staindf = scdfs.groupby(["Metadata_Plate", "Metadata_Well", "Metadata_Stain"]).agg(agg_funcs).reset_index()

In [12]:
print(staindf)

   Metadata_Plate Metadata_Well Metadata_Stain Metadata_Condition  \
0          slide1            A1           A647                NTC   
1          slide1            A1           DAPI                NTC   
2          slide1            A1           GOLD                NTC   
3          slide1            A2           A647            ALY kd8   
4          slide1            A2           DAPI            ALY kd8   
..            ...           ...            ...                ...   
91         slide4            B3           DAPI        TMEM259 kd5   
92         slide4            B3           GOLD        TMEM259 kd5   
93         slide4            B4           A647          untreated   
94         slide4            B4           DAPI          untreated   
95         slide4            B4           GOLD          untreated   

    Metadata_Cell_Count  Nuclei_Granularity_1  \
0                  1101              0.171474   
1                  1101             -0.217538   
2                  1101 

# MIC Comparisons
Compares MIC scores between DAPI and nuclear speckle stains of the same well.

In [13]:
micdfs = []
speckle_stains = {"GOLD", "A647"}

for stain in speckle_stains:
    for feature_order in ("mic", "shuffled_mic"):

        if feature_order == "mic":
            mic_comparator = MIC()

        else:
            # Shuffles the samples/features depending on your perspective
            mic_comparator = ShuffledMIC()

        speckle_staindf = staindf.loc[staindf["Metadata_Stain"] != stain]

        comparer = PairwiseCompare(
            _df=speckle_staindf,
            _comparator=mic_comparator,
            _antehoc_group_cols=["Metadata_Plate", "Metadata_Well", "Metadata_Condition"],
            _posthoc_group_cols=["Metadata_Stain"],
            _feat_cols=feat_cols,
        )

        comparer.intra_comparisons()

        micdf = pd.DataFrame(mic_comparator.comparisons)
        micdf = micdf.assign(Metadata_Comparison_Type=feature_order)
        micdfs.append(micdf)

micdfs = pd.concat(micdfs, axis=0)

In [14]:
agg_merge_cols = ["Metadata_Plate", "Metadata_Well"]

micdfs = pd.merge(
    left=micdfs,
    right=staindf[agg_merge_cols + ["Metadata_Cell_Count", "Metadata_Condition"]],
    how="inner",
    left_on=[
        "Metadata_Plate__antehoc_group0",
        "Metadata_Well__antehoc_group0"
    ],
    right_on=agg_merge_cols,
)

In [15]:
print(micdfs)

        mic_e Metadata_Plate__antehoc_group0 Metadata_Plate__antehoc_group1  \
0    0.857133                         slide1                         slide1   
1    0.857133                         slide1                         slide1   
2    0.857133                         slide1                         slide1   
3    0.152908                         slide1                         slide1   
4    0.152908                         slide1                         slide1   
..        ...                            ...                            ...   
379  0.726352                         slide4                         slide4   
380  0.726352                         slide4                         slide4   
381  0.170285                         slide4                         slide4   
382  0.170285                         slide4                         slide4   
383  0.170285                         slide4                         slide4   

    Metadata_Well__antehoc_group0 Metadata_Well__an

# Save Results

In [16]:
for stain in speckle_stains:

    pattern = f"DAPI|{re.escape(stain)}"
    stain_micdfs = micdfs.loc[micdfs["Metadata_Stain__posthoc_group0"].str.contains(pattern, regex=True) & micdfs["Metadata_Stain__posthoc_group1"].str.contains(pattern, regex=True)]
    print(stain_micdfs)

    sns.histplot(data=stain_micdfs, x="mic_e", hue="Metadata_Comparison_Type",
    palette={"shuffled_mic": 'blue', "mic": 'red'}, bins=10, kde=False)

    plt.gcf().set_size_inches(18, 10)

    plt.xlabel("MIC", fontsize=13)
    plt.ylabel("Density", fontsize=13)

    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    plt.title(f"Distributions of Maximal Information Coeficient (MIC)\nBetween DAPI and {stain} Features per Well", fontsize=16)

    plt.xlim(0, 1)
    plt.ylim(0.0, 80.0)

    plt.savefig(distribution_figures_path / f"mic_distributions_dapi_{stain.lower()}_wells.png")
    plt.close()

        mic_e Metadata_Plate__antehoc_group0 Metadata_Plate__antehoc_group1  \
6    0.857133                         slide1                         slide1   
7    0.857133                         slide1                         slide1   
8    0.857133                         slide1                         slide1   
9    0.195190                         slide1                         slide1   
10   0.195190                         slide1                         slide1   
..        ...                            ...                            ...   
379  0.726352                         slide4                         slide4   
380  0.726352                         slide4                         slide4   
381  0.170285                         slide4                         slide4   
382  0.170285                         slide4                         slide4   
383  0.170285                         slide4                         slide4   

    Metadata_Well__antehoc_group0 Metadata_Well__an

        mic_e Metadata_Plate__antehoc_group0 Metadata_Plate__antehoc_group1  \
0    0.857133                         slide1                         slide1   
1    0.857133                         slide1                         slide1   
2    0.857133                         slide1                         slide1   
3    0.152908                         slide1                         slide1   
4    0.152908                         slide1                         slide1   
..        ...                            ...                            ...   
373  0.813132                         slide4                         slide4   
374  0.813132                         slide4                         slide4   
375  0.152928                         slide4                         slide4   
376  0.152928                         slide4                         slide4   
377  0.152928                         slide4                         slide4   

    Metadata_Well__antehoc_group0 Metadata_Well__an

In [17]:
micdfs.to_parquet(mic_comparisons_path / "well_sirna_mic_comparisons.parquet")